# __Plot neighbourhood probabilities (precip percentiles)__

In [ ]:
%env MIR_GRIB_INPUT_BUFFER_SIZE=7688961960
%env MARS_READANY_BUFFER_SIZE=7688961960
%env MIR_CACHE_PATH=${SCRATCH}

In [ ]:
import metview as mv
import numpy as np
from datetime import date, datetime, timedelta
import pandas as pd
import numpy as np
import os
import csv
import math

In [ ]:
use_mars = False    # download oper ENS data from MARS?

# Plot parameters
my_area=[75, -15, 20, 60]     # North/West/South/East
adate = datetime(2025,4,10)   # Forecast initialisation day
my_step = 36           # forecast lead time
prec_acc=6             # precip accumulation in hours
my_percentile = 99      # percentile


In [ ]:
path_in_EDT = "/ec/vol/destine/neighbourhood/edt/"
path_in_ENS = "/ec/vol/destine/neighbourhood/oper_ENS_9km/"
path_in_NB = "/ec/vol/destine/neighbourhood/percentile/"

In [ ]:
path_file_in1cf = adate.strftime("%Y%m%d")  + "_cf.grib" 
path_file_in1pf = adate.strftime("%Y%m%d")  + "_pf.grib" 
path_file_in1edt = adate.strftime("%Y%m%d")  + ".grib" 

In [ ]:
# download oper ENS data from MARS
if use_mars:
    ret_field = mv.retrieve(
        date= adate,
        time= 0,
        step= [0,"to",120,"by",6],
        type= "pf",
        number=[1,"to",50],
        class_= "od",
        stream= "enfo",
        levtype= "sfc",
        expver= 1,
        param="228.128"     # total precipitation
        
    )

    # save to file
    print("writing retrieved file: ", path_in_ENS + path_file_in1pf)
    mv.write(path_in_ENS + path_file_in1pf,ret_field)

    ret_field = mv.retrieve(
        date= adate,
        time= 0,
        step= [0,"to",120,"by",6],
        type= "cf",
        class_= "od",
        stream= "enfo",
        levtype= "sfc",
        expver= 1,
        param="228.128"     # total precipitation
        
    )

    # save to file
    print("writing retrieved file: ", path_in_ENS + path_file_in1cf)
    mv.write(path_in_ENS + path_file_in1cf,ret_field)

In [ ]:
file_grib1cf = mv.read(path_in_ENS + path_file_in1cf)
file_grib1pf = mv.read(path_in_ENS + path_file_in1pf)
file_grib1 = mv.merge(file_grib1cf, file_grib1pf)
file_grib1edt = mv.read(path_in_EDT + path_file_in1edt)

# Plot DestinE forecast

In [ ]:
var_2 = file_grib1edt.select(
    shortName="tp",
    stepRange=my_step,
    )

var_1 = file_grib1edt.select(
    shortName="tp",
    stepRange=my_step-prec_acc,
    )

In [ ]:
var=(var_2-var_1)*1000.

In [ ]:
max(var.values())

In [ ]:
# set up the geographical area
my_view = mv.geoview(
    map_area_definition="CORNERS", map_projection="CYLINDRICAL", area=my_area  # North/West/South/East
)

# set up the coastlines
my_coast = mv.mcoast(
    map_coastline_land_shade="ON", map_coastline_land_shade_colour="CREAM"
)

# set up contour
my_contour = mv.mcont(
    legend                       = "on",
    contour                      = "off",
    contour_level_selection_type = "level_list",
    contour_level_list           = [-10,0.5,2,5,10,20,30,40,50,60,80,100,125,150,200,300,500,10000],
    contour_label                = "off",
    contour_shade                = "on",
    contour_shade_colour_method  = "list",
    contour_shade_method         = "area_fill",
    contour_shade_colour_list    = ["white","RGB(0.75,0.95,0.93)","RGB(0.45,0.93,0.78)","RGB(0.06999,0.85,0.61)","RGB(0.53,0.8,0.13)","RGB(0.6,0.91,0.05699)","RGB(0.9,1,0.4)","RGB(0.89,0.89,0.066)","RGB(1,0.73,0.003906)","RGB(1,0.49,0.003906)","red","RGB(0.85,0.003906,1)","RGB(0.63,0.007294,0.92)","RGB(0.37,0.29,0.91)","RGB(0.03999,0.03999,0.84)","RGB(0.04199,0.04199,0.43)","RGB(0.45,0.45,0.45)"]
    )

# set-up the title
my_title = mv.mtext(
    text_font_size=0.50,
    text_lines=[
        "<font size='0.7'>DestinE 4.4 km, "+str(prec_acc)+"h prec. [mm] </font>",
        "<magics_title/>",
        "",
    ],
    text_colour="CHARCOAL",
)

my_legend = mv.mlegend(
    legend_text_font_size = 0.4,
)

# define the output plot file
# mv.setoutput(mv.pdf_output(output_name="contour9"))

# plot the data onto the map
mv.plot(my_view, my_coast, var, my_contour, my_title, my_legend)
#mv.plot( my_coast, var, my_contour, my_title)


# Plot NB probability field (percentile)

In [ ]:
file_in = path_in_NB + "NB_tp6h_p" + str(float(my_percentile)) + "_rVR_ST0_" + adate.strftime("%Y%m%d") + "_step_6-120.grib2"
file_grib_NB = mv.read(file_in)

In [ ]:
file_in

In [ ]:
var_NB = file_grib_NB.select(
    shortName="tp",
    stepRange=str(my_step-prec_acc)+"-"+str(my_step),
    )

In [ ]:
max(var_NB.values())

In [ ]:
# set up the geographical area
my_view = mv.geoview(
    map_area_definition="CORNERS", map_projection="CYLINDRICAL", area=my_area  # North/West/South/East
)

# set up the coastlines
my_coast = mv.mcoast(
    map_coastline_land_shade="ON", map_coastline_land_shade_colour="CREAM"
)

# set up contour
my_contour = mv.mcont(
    legend                       = "on",
    contour                      = "off",
    contour_level_selection_type = "level_list",
    contour_level_list           = [-10,0.5,2,5,10,20,30,40,50,60,80,100,125,150,200,300,500,10000],
    contour_label                = "off",
    contour_shade                = "on",
    contour_shade_colour_method  = "list",
    contour_shade_method         = "area_fill",
    contour_shade_colour_list    = ["white","RGB(0.75,0.95,0.93)","RGB(0.45,0.93,0.78)","RGB(0.06999,0.85,0.61)","RGB(0.53,0.8,0.13)","RGB(0.6,0.91,0.05699)","RGB(0.9,1,0.4)","RGB(0.89,0.89,0.066)","RGB(1,0.73,0.003906)","RGB(1,0.49,0.003906)","red","RGB(0.85,0.003906,1)","RGB(0.63,0.007294,0.92)","RGB(0.37,0.29,0.91)","RGB(0.03999,0.03999,0.84)","RGB(0.04199,0.04199,0.43)","RGB(0.45,0.45,0.45)"]
    )

# set-up the title
my_title = mv.mtext(
    text_font_size=0.50,
    text_lines=[
        "<font size='0.7'>DestinE 4.4 km, NB "+str(my_percentile)+" Percentile for "+str(prec_acc)+"h prec. [mm]  </font>",
        "<magics_title/>",
        "",
    ],
    text_colour="CHARCOAL",
)

my_legend = mv.mlegend(
    legend_text_font_size = 0.4,
)

# define the output plot file
# mv.setoutput(mv.pdf_output(output_name="contour9"))

# plot the data onto the map
mv.plot(my_view, my_coast, var_NB/1, my_contour, my_title, my_legend)
#mv.plot( my_coast, var_NB/1, my_contour, my_title)


# __Plot ENS percentiles__

In [ ]:
var_ENS_2 = file_grib1.select(
    shortName="tp",
    stepRange=my_step,
    )

In [ ]:
var_ENS_1 = file_grib1.select(
    shortName="tp",
    stepRange=my_step-prec_acc,
    )

In [ ]:
var_ENS = (var_ENS_2 - var_ENS_1) * 1000

In [ ]:
ENS_percentile = mv.percentile(data=var_ENS, percentiles=my_percentile)

In [ ]:
# set up the geographical area
my_view = mv.geoview(
    map_area_definition="CORNERS", map_projection="CYLINDRICAL", area=my_area  # North/West/South/East
)

# set up the coastlines
my_coast = mv.mcoast(
    map_coastline_land_shade="ON", map_coastline_land_shade_colour="CREAM"
)

# set up contour
my_contour = mv.mcont(
    legend                       = "on",
    contour                      = "off",
    contour_level_selection_type = "level_list",
    contour_level_list           = [-10,0.5,2,5,10,20,30,40,50,60,80,100,125,150,200,300,500,10000],
    contour_label                = "off",
    contour_shade                = "on",
    contour_shade_colour_method  = "list",
    contour_shade_method         = "area_fill",
    contour_shade_colour_list    = ["white","RGB(0.75,0.95,0.93)","RGB(0.45,0.93,0.78)","RGB(0.06999,0.85,0.61)","RGB(0.53,0.8,0.13)","RGB(0.6,0.91,0.05699)","RGB(0.9,1,0.4)","RGB(0.89,0.89,0.066)","RGB(1,0.73,0.003906)","RGB(1,0.49,0.003906)","red","RGB(0.85,0.003906,1)","RGB(0.63,0.007294,0.92)","RGB(0.37,0.29,0.91)","RGB(0.03999,0.03999,0.84)","RGB(0.04199,0.04199,0.43)","RGB(0.45,0.45,0.45)"]
    )

# set-up the title
my_title = mv.mtext(
    text_font_size=0.50,
    text_lines=[
        "<font size='0.7'>ENS 9 km, "+str(my_percentile)+" Percentile for "+str(prec_acc)+"h prec. [mm]   </font>",
        "<magics_title/>",
        "",
    ],
    text_colour="CHARCOAL",
)

my_legend = mv.mlegend(
    legend_text_font_size = 0.4,
)

mv.plot(my_view, my_coast, ENS_percentile, my_contour, my_title, my_legend)
#mv.plot( my_coast, prob*100., my_contour, my_title)
